* Temperature (F) vs. Latitude
    * Temperature goes down as latitude goes up
* Humidity (%) vs. Latitude
    * There doesn't seem to be a relationship between humidity and latitude
* Cloudiness (%) vs. Latitude
    * There doesnt seem to be a relationship between clouds and latitude 
* Wind Speed (mph) vs. Latitude
    * Wind speeds drift higher as the latitude gets higher. 

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json 

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [22]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

608

## Perform API Calls

In [23]:
citypd = pd.DataFrame({
    'City': cities
})

citypd.head()

,City
0,sioux lookout
1,kaitangata
2,kodiak
3,ponta delgada
4,hobart


In [24]:
# OpenWeatherMap API Key
api_key = api_keys.api_key



In [ ]:
column_names = ["City","Country","Temperature (F)",
                "Latitude","Longitude","Humidity (%)",
                "Cloudiness (%)", "Wind Speed (mph)"]
cleancitypd = pd.DataFrame(columns=column_names)

# Loop through the list of cities and add the data to the df.
url = "http://api.openweathermap.org/data/2.5/weather?"

row_count = 1

for index, row in citypd.iterrows():
    target_city = row["City"]
    target_url = url + 'appid=' + api_key + '&units=IMPERIAL' + '&q=' + target_city.replace(' ',"+")
    weather_data = requests.get(target_url).json()
    if weather_data["cod"] == "404":
        print("City not found, skipping...")

    else:
        cleancitypd.set_value(index, "City", weather_data["name"])
        cleancitypd.set_value(index, "Country", weather_data["sys"]["country"])
        cleancitypd.set_value(index, "Temperature (F)", weather_data["main"]["temp"])
        cleancitypd.set_value(index, "Latitude", weather_data["coord"]["lat"])
        cleancitypd.set_value(index, "Longitude", weather_data["coord"]["lon"])
        cleancitypd.set_value(index, "Humidity (%)", weather_data["main"]["humidity"])
        cleancitypd.set_value(index, "Cloudiness (%)", weather_data["clouds"]["all"])
        cleancitypd.set_value(index, "Wind Speed (mph)", weather_data["wind"]["speed"])
        
        print("------------------------")
        print("Procesing: City # " , row_count, ' | ' , weather_data["name"], weather_data["sys"]["country"])
        print(target_url)
        row_count += 1

C:\Users\molli\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\molli\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\molli\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\molli\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\molli\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: set_value is deprecated and will be removed in a future rele

------------------------
Procesing: City #  1  |  Sioux Lookout CA
http://api.openweathermap.org/data/2.5/weather?appid=870f8ebcb7593b1b710e4746bf96e846&units=IMPERIAL&q=sioux+lookout
------------------------
Procesing: City #  2  |  Kaitangata NZ
http://api.openweathermap.org/data/2.5/weather?appid=870f8ebcb7593b1b710e4746bf96e846&units=IMPERIAL&q=kaitangata
------------------------
Procesing: City #  3  |  Kodiak US
http://api.openweathermap.org/data/2.5/weather?appid=870f8ebcb7593b1b710e4746bf96e846&units=IMPERIAL&q=kodiak
------------------------
Procesing: City #  4  |  Ponta Delgada PT
http://api.openweathermap.org/data/2.5/weather?appid=870f8ebcb7593b1b710e4746bf96e846&units=IMPERIAL&q=ponta+delgada
------------------------
Procesing: City #  5  |  Hobart AU
http://api.openweathermap.org/data/2.5/weather?appid=870f8ebcb7593b1b710e4746bf96e846&units=IMPERIAL&q=hobart
------------------------
Procesing: City #  6  |  Norman Wells CA
http://api.openweathermap.org/data/2.5/weather?a

In [ ]:
cleancitypd.to_csv("Weather_Data.csv")
cleancitypd.head()

In [ ]:
plt.scatter(cleancitypd["Latitude"], 
            cleancitypd["Temperature (F)"], c=cleancitypd["Temperature (F)"],
            edgecolor="black", linewidths=1, marker="o", 
            cmap='plasma', alpha=0.8, label="City")

plt.style.use('seaborn')
plt.title(f"City Latitude vs. Max Temperature")
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)


plt.savefig("Latitude_Temperature.png")

plt.show()


In [ ]:
plt.scatter(cleancitypd["Latitude"], 
            cleancitypd["Humidity (%)"], c=cleancitypd["Humidity (%)"],
            edgecolor="black", linewidths=1, marker="o", 
            cmap='GnBu', alpha=0.8, label="City")

plt.style.use('seaborn')
plt.title(f"City Latitude vs. Humidity (%)")
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)


plt.savefig("Latitude_Humidity.png")

plt.show()


In [ ]:
plt.scatter(cleancitypd["Latitude"], 
            cleancitypd["Cloudiness (%)"], c=cleancitypd["Cloudiness (%)"],
            edgecolor="black", linewidths=1, marker="o", 
            cmap='PuBuGn', alpha=0.8, label="City")

plt.style.use('seaborn')
plt.title(f"City Latitude vs. Cloudiness (%)")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)


plt.savefig("Latitude_Cloud.png")

plt.show()

In [ ]:
plt.scatter(cleancitypd["Latitude"], 
            cleancitypd["Wind Speed (mph)"], c=cleancitypd["Wind Speed (mph)"],
            edgecolor="black", linewidths=1, marker="o", 
            cmap='cividis', alpha=0.8, label="City")

plt.style.use('seaborn')
plt.title(f"City Latitude vs. Wind Speed (mph)")
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)


plt.savefig("Latitude_Wind.png")

plt.show()